In [ ]:
# http 通信用ライブラリ
from requests_html import HTMLSession
# html 処理用ライブラリ
from bs4 import BeautifulSoup
# 正規表現用ライブラリ
import re


# ログインするサイトの場合、cookie の処理が必要となるが
# session オブジェクトで操作するとその辺の面倒さがなくなってお手軽
session = HTMLSession()

In [ ]:
# ニュース一覧を取得する
base_url = "https://www.msn.com"

# MSN のニュースのトップにアクセスします。
# response オブジェクトの詳細は https://requests-docs-ja.readthedocs.io/en/latest/user/quickstart/#id3 に記載されています
response = session.get(base_url + "/ja-jp/news")
print(response.status_code)
print(response.text)

In [ ]:
# html をパースする
bs = BeautifulSoup(response.text, "lxml")

# news 一覧を作成
news_list = []
# すべての A タグを取得し、ループで処理
for a_tag in bs.find_all("a"):
    # 遷移先を取得
    href = a_tag.get("href")
    # 記事のタイトルを取得 ( 不要な空白と制御文字を削除 )
    title = a_tag.get_text().strip().strip('\r\n\t')
    # 記事の URL だった場合リストに追加
    if re.match("/ja-jp/news/[^/]*/.*" , href) :
        news_list.append({"href":href, "title":title})

print(news_list)

In [ ]:
# ディレクトリを作成するため、ライブラリを読み込む
import os
import os.path

dir = "news"

if not os.path.isdir(dir):
    os.mkdir(dir)

# 記事本文を取得しファイルに書き込む
for dict in news_list:
    response = session.get(base_url + dict["href"])
    bs = BeautifulSoup(response.text, "html.parser")
    
    # title タグからタイトルを取得
    title = bs.title.get_text()
    # 記事の公開日時が yyyy-MM-ddThh:mm:ss.000Z なので数字のみにする
    datetime = bs.select("time")[0].get("datetime").translate(str.maketrans("", "", "-T:.Z"))
    # タイトルから禁則文字を除いてファイル名にする
    path = dir+"/"+title.translate(str.maketrans("", "", " 　¥\/:*?\"<>|."))+"_"+datetime+".txt"
    with open(path, mode='w') as f:
        # 本文を書き込む
        for p in bs.select("section .articlebody p"):
            f.write(p.get_text().strip())